<a href="https://colab.research.google.com/github/unt-iialab/INFO5731_Spring2020/blob/master/Assignments/INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment. (Use 500 records amount all the data you collected).

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as the 500 reviews (abstracts, or tweets). 


In [97]:
# Write your code here

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use(style='seaborn')
from nltk.util import ngrams
import nltk, re, string, collections
from nltk.corpus import stopwords
import spacy
nlp = spacy.load("en_core_web_sm")
%matplotlib inline



In [98]:
df=pd.read_csv('processed.csv',encoding = "ISO-8859-1")
df.head()

,text,sentiment,character_cnt,word_counts,characters_per_word,special_characters,num,result
0,Wow. Yall needa step it up @Apple RT @heynyla:...,-1,105,20,5.250000,2,0,Negative
1,What Happened To Apple Inc? http://t.co/FJEX...,0,101,10,10.100000,1,0,Neutral
2,Thank u @apple I can now compile all of the pi...,1,108,19,5.684211,1,0,Positive
3,The oddly uplifting story of the Apple co-foun...,0,115,17,6.764706,0,0,Neutral
4,@apple can i exchange my iphone for a differen...,0,77,15,5.133333,1,0,Neutral


In [99]:
stop = stopwords.words('english')
df['text'] = df['text'].apply(lambda x: x.replace(r'"', ''))
df['text'] = df['text'].apply(
    lambda x: x.replace(r'[\n\n\n\n\n\n\n\n\n\n  \n  \n    , \n  ', ''))
df['text'] = df['text'].apply(
    lambda x: x.replace(r'\n, \n  \n]', ''))
df['text'] = df['text'].apply(lambda x: x.lower())
df.text = df.text.apply(lambda x: re.sub(r'\w*\d', '', x))
df.text = df.text.str.replace('[^\w\s]', '')
df.text = df.text.apply(
    lambda x: ' '.join(x for x in x.split() if x not in stop))
df.drop(['sentiment', 'character_cnt','word_counts','characters_per_word','special_characters','num','result'], axis=1, inplace=True)


C:\Users\stech\AppData\Local\Temp/ipykernel_8916/3312584693.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df.text = df.text.str.replace('[^\w\s]', '')


In [100]:
df.head()

,text
0,wow yall needa step apple rt heynyla music sna...
1,happened apple inc httptcou aapl apple moneypr...
2,thank u apple compile pics communicate one pla...
3,oddly uplifting story apple cofounder sold sta...
4,apple exchange iphone different color lmao cha...


In [101]:
text = df.text.values
text = ' '.join([t for t in text])
text

'wow yall needa step apple rt heynyla music snapchat damn time thank happened apple inc httptcou aapl apple moneypress httptcowxkmmtmarw thank u apple compile pics communicate one place httptcooya oddly uplifting story apple cofounder sold stake aapl aapl httptcopj apple exchange iphone different color lmao changed mind rt jpdesloges apple acted unfairly suppressing digital music competition paul kedrosky aapl aapl httptco lets forget press effects stock apple defend ipod antitrust suit httptco apple inc deleted songs rival services ipods httptcox aapl teamcavuto apple theyre staging die ar apple store iphone happy monday camera fancy apple plus suddenly stopped working weekend instead meme httptcoxdysgcxuuv facebook ceo mark zuckerberg criticizes apple aapl aapl httptcoevrsnvdexe fuck apple apple need sort phones onerepublic apple feet keep flying markzuckerbergf facebook tim cook apple sparring plz talk real connectivity web httptcox apple youre shit fix problem apple supplier skywor

In [102]:
tokens = nltk.word_tokenize(text)
tokens

['wow',
 'yall',
 'needa',
 'step',
 'apple',
 'rt',
 'heynyla',
 'music',
 'snapchat',
 'damn',
 'time',
 'thank',
 'happened',
 'apple',
 'inc',
 'httptcou',
 'aapl',
 'apple',
 'moneypress',
 'httptcowxkmmtmarw',
 'thank',
 'u',
 'apple',
 'compile',
 'pics',
 'communicate',
 'one',
 'place',
 'httptcooya',
 'oddly',
 'uplifting',
 'story',
 'apple',
 'cofounder',
 'sold',
 'stake',
 'aapl',
 'aapl',
 'httptcopj',
 'apple',
 'exchange',
 'iphone',
 'different',
 'color',
 'lmao',
 'changed',
 'mind',
 'rt',
 'jpdesloges',
 'apple',
 'acted',
 'unfairly',
 'suppressing',
 'digital',
 'music',
 'competition',
 'paul',
 'kedrosky',
 'aapl',
 'aapl',
 'httptco',
 'lets',
 'forget',
 'press',
 'effects',
 'stock',
 'apple',
 'defend',
 'ipod',
 'antitrust',
 'suit',
 'httptco',
 'apple',
 'inc',
 'deleted',
 'songs',
 'rival',
 'services',
 'ipods',
 'httptcox',
 'aapl',
 'teamcavuto',
 'apple',
 'theyre',
 'staging',
 'die',
 'ar',
 'apple',
 'store',
 'iphone',
 'happy',
 'monday',
 'c

In [103]:
def find_n_grams(tokens, order):
    t = ngrams(tokens, order)
    t_freq = collections.Counter(t)
    return t_freq

In [104]:
bigrams = find_n_grams(tokens, 2)
bigrams

Counter({('wow', 'yall'): 1,
         ('yall', 'needa'): 3,
         ('needa', 'step'): 1,
         ('step', 'apple'): 2,
         ('apple', 'rt'): 20,
         ('rt', 'heynyla'): 1,
         ('heynyla', 'music'): 1,
         ('music', 'snapchat'): 1,
         ('snapchat', 'damn'): 1,
         ('damn', 'time'): 1,
         ('time', 'thank'): 1,
         ('thank', 'happened'): 1,
         ('happened', 'apple'): 1,
         ('apple', 'inc'): 50,
         ('inc', 'httptcou'): 1,
         ('httptcou', 'aapl'): 1,
         ('aapl', 'apple'): 23,
         ('apple', 'moneypress'): 1,
         ('moneypress', 'httptcowxkmmtmarw'): 1,
         ('httptcowxkmmtmarw', 'thank'): 1,
         ('thank', 'u'): 3,
         ('u', 'apple'): 12,
         ('apple', 'compile'): 1,
         ('compile', 'pics'): 1,
         ('pics', 'communicate'): 1,
         ('communicate', 'one'): 1,
         ('one', 'place'): 1,
         ('place', 'httptcooya'): 1,
         ('httptcooya', 'oddly'): 1,
         ('oddly', 'up

In [105]:
trigrams = find_n_grams(tokens, 3)
trigrams

Counter({('wow', 'yall', 'needa'): 1,
         ('yall', 'needa', 'step'): 1,
         ('needa', 'step', 'apple'): 1,
         ('step', 'apple', 'rt'): 1,
         ('apple', 'rt', 'heynyla'): 1,
         ('rt', 'heynyla', 'music'): 1,
         ('heynyla', 'music', 'snapchat'): 1,
         ('music', 'snapchat', 'damn'): 1,
         ('snapchat', 'damn', 'time'): 1,
         ('damn', 'time', 'thank'): 1,
         ('time', 'thank', 'happened'): 1,
         ('thank', 'happened', 'apple'): 1,
         ('happened', 'apple', 'inc'): 1,
         ('apple', 'inc', 'httptcou'): 1,
         ('inc', 'httptcou', 'aapl'): 1,
         ('httptcou', 'aapl', 'apple'): 1,
         ('aapl', 'apple', 'moneypress'): 1,
         ('apple', 'moneypress', 'httptcowxkmmtmarw'): 1,
         ('moneypress', 'httptcowxkmmtmarw', 'thank'): 1,
         ('httptcowxkmmtmarw', 'thank', 'u'): 1,
         ('thank', 'u', 'apple'): 3,
         ('u', 'apple', 'compile'): 1,
         ('apple', 'compile', 'pics'): 1,
         ('co

In [106]:
def calculate_freq(bigrams, text):
    return [v / text.count(k[0]) for k, v in bigrams.items()]

In [107]:
_ = [print(k, freq) for k, freq in zip(bigrams, calculate_freq(bigrams, text))]

('wow', 'yall') 0.1111111111111111
('yall', 'needa') 0.10714285714285714
('needa', 'step') 0.3333333333333333
('step', 'apple') 0.25
('apple', 'rt') 0.011235955056179775
('rt', 'heynyla') 0.0027548209366391185
('heynyla', 'music') 1.0
('music', 'snapchat') 0.02631578947368421
('snapchat', 'damn') 0.3333333333333333
('damn', 'time') 0.1
('time', 'thank') 0.014925373134328358
('thank', 'happened') 0.014492753623188406
('happened', 'apple') 0.08333333333333333
('apple', 'inc') 0.028089887640449437
('inc', 'httptcou') 0.009009009009009009
('httptcou', 'aapl') 0.029411764705882353
('aapl', 'apple') 0.043560606060606064
('apple', 'moneypress') 0.0005617977528089888
('moneypress', 'httptcowxkmmtmarw') 1.0
('httptcowxkmmtmarw', 'thank') 1.0
('thank', 'u') 0.043478260869565216
('u', 'apple') 0.004928131416837783
('apple', 'compile') 0.0005617977528089888
('compile', 'pics') 1.0
('pics', 'communicate') 0.3333333333333333
('communicate', 'one') 1.0
('one', 'place') 0.001984126984126984
('place', 

('monday', 'december') 0.5
('december', 'httptcouarms') 0.02
('httptcouarms', 'britt_bumgarner') 1.0
('britt_bumgarner', 'briannalengy') 1.0
('briannalengy', 'mean') 1.0
('mean', 'didnt') 0.1111111111111111
('didnt', 'auto') 0.06666666666666667
('auto', 'correct') 0.17647058823529413
('correct', 'really') 0.07692307692307693
('really', 'fault') 0.038461538461538464
('fault', 'apple') 0.3333333333333333
('aapl', 'buy') 0.003787878787878788
('buy', 'reiterated') 0.05714285714285714
('reiterated', 'target') 1.0
('target', 'raised') 0.07142857142857142
('raised', 'bank') 0.14285714285714285
('bank', 'america') 0.16666666666666666
('america', 'merrill') 1.0
('merrill', 'lynch') 0.5
('lynch', 'aapl') 1.0
('aapl', 'brief') 0.003787878787878788
('brief', 'explanation') 1.0
('explanation', 'httpstcov') 0.5
('httpstcov', 'aapl') 0.5
('fuck', 'type') 0.006993006993006993
('type', 'shit') 0.16666666666666666
('shit', 'apple') 0.175
('apple', 'autocorrect') 0.0016853932584269663
('autocorrect', 'si

('million', 'auditorium') 0.1111111111111111
('auditorium', 'httptcoqklb') 0.5
('httptcoqklb', 'aapl') 1.0
('aapl', 'wwdc') 0.001893939393939394
('wwdc', 'httptcoqyhvucusyp') 1.0
('httptcoqyhvucusyp', 'youll') 1.0
('youll', 'able') 0.3333333333333333
('able', 'learn') 0.02040816326530612
('learn', 'code') 0.16666666666666666
('code', 'free') 0.05263157894736842
('free', 'next') 0.02127659574468085
('next', 'week') 0.125
('week', 'nearest') 0.03125
('nearest', 'apple') 1.0
('store', 'httptcox') 0.011764705882352941
('httptcox', 'find') 0.037037037037037035
('find', 'alarming') 0.125
('alarming', 'apple') 1.0
('apple', 'power') 0.0022471910112359553
('power', 'cords') 0.14285714285714285
('cords', 'quickly') 0.16666666666666666
('quickly', 'fray') 0.5
('fray', 'dont') 0.5
('dont', 'always') 0.02127659574468085
('always', 'notice') 0.25
('notice', 'dont') 0.25
('dont', 'travel') 0.02127659574468085
('travel', 'applecordfire') 0.3333333333333333
('applecordfire', 'best') 0.25
('best', 'ste

('alarm', 'fails') 0.125
('fails', 'work') 1.0
('work', 'sometimes') 0.014285714285714285
('sometimes', 'missed') 0.2
('missed', 'bonus') 1.0
('bonus', 'points') 0.3333333333333333
('points', 'class') 0.5
('class', 'sooooo') 0.16666666666666666
('sooooo', 'annoyed') 0.6666666666666666
('annoyed', 'would') 0.25
('would', 'nice') 0.045454545454545456
('nice', 'could') 0.08333333333333333
('could', 'type') 0.03333333333333333
('type', 'one') 0.16666666666666666
('one', 'sentence') 0.001984126984126984
('sentence', 'without') 0.25
('without', 'computer') 0.08333333333333333
('computer', 'freezing') 0.06896551724137931
('freezing', 'apple') 0.3333333333333333
('apple', 'microsoft') 0.0011235955056179776
('microsoft', 'word') 0.25
('word', 'internet') 0.047619047619047616
('internet', 'everyone') 0.16666666666666666
('everyone', 'apple') 0.25
('apple', 'still') 0.0022471910112359553
('still', 'struggles') 0.034482758620689655
('struggles', 'meet') 1.0
('meet', 'iphone') 0.3333333333333333
('

('aapl', 'drm') 0.001893939393939394
('drm', 'trial') 0.5
('trial', 'finally') 0.0625
('finally', 'underway') 0.1
('underway', 'httpstcomvjwhtnq') 0.3333333333333333
('httpstcomvjwhtnq', 'yosemite') 1.0
('yosemite', 'update') 0.08333333333333333
('update', 'annoying') 0.01818181818181818
('annoying', 'regret') 0.1111111111111111
('regret', 'installing') 1.0
('installing', '_') 1.0
('_', 'apple') 0.018518518518518517
('apple', 'sort') 0.0016853932584269663
('sort', 'shit') 0.09090909090909091
('shit', 'bro') 0.0125
('bro', 'rt') 0.034482758620689655
('rt', 'willliamlemar') 0.0027548209366391185
('willliamlemar', 'rt') 1.0
('rt', 'justpoetic_') 0.005509641873278237
('justpoetic_', 'fuck') 1.0
('fuck', 'recently') 0.02097902097902098
('deleted', 'photo') 0.13636363636363635
('photo', 'album') 0.17647058823529413
('album', 'apple') 0.375
('apple', 'know') 0.0016853932584269663
('strong', 'supply') 0.125
('supply', 'chain') 0.2
('chain', 'makes') 0.5
('makes', 'way') 0.09090909090909091
('w

...

('httptcoyawto', 'apple') 1.0
('thanks', 'making') 0.02127659574468085
('making', 'shittiest') 0.0625
('shittiest', 'chargers') 1.0
('chargers', 'trademark') 0.041666666666666664
('trademark', 'confusion') 0.3333333333333333
('confusion', 'apple') 1.0
('store', 'trademark') 0.011764705882352941
('trademark', 'rejected') 0.3333333333333333
('rejected', 'australia') 0.5
('australia', 'approved') 0.3333333333333333
('approved', 'europe') 0.5
('europe', 'amphttptcoeasponshkf') 1.0
('amphttptcoeasponshkf', 'congrats') 1.0
('apple', 'mary') 0.0005617977528089888
('mary', 'sotos') 0.3333333333333333
('sotos', 'worldresources') 1.0
('worldresources', 'green') 1.0
('green', 'power') 0.2
('power', 'leadership') 0.07142857142857142
('leadership', 'awards') 1.0
('awards', 'greenemarktplc') 0.3333333333333333
('greenemarktplc', 'httptcosf') 1.0
('httptcosf', 'activation') 0.5
('activation', 'airdrop') 1.0
('airdrop', 'ios') 1.0
('ios', 'kills') 0.01818181818181818
('kills', 'wifi') 1.0
('wifi', 'do

('repeatinghelp', 'apple') 1.0
('httptco', 'fulltime') 0.0012195121951219512
('fulltime', 'trader') 0.5
('trader', 'trading') 0.2
('esignal', 'httptcokgb') 0.3333333333333333
('httptcokgb', 'rt') 0.3333333333333333
('rt', 'peterpham') 0.0027548209366391185
('peterpham', 'bought') 1.0
('bought', 'augustsmartlock') 0.14285714285714285
('augustsmartlock', 'apple') 1.0
('apple', 'storepretty') 0.0005617977528089888
('storepretty', 'good') 1.0
('good', 'logo') 0.043478260869565216
('logo', 'match') 0.3333333333333333
('match', 'cant') 0.3333333333333333
('wait', 'install') 0.05
('install', 'httptco') 0.1
('httptco', 'king') 0.0012195121951219512
('phablets', 'apples') 0.5
('plus', 'bestseller') 0.03125
('bestseller', 'past') 1.0
('past', 'three') 0.2
('three', 'months') 0.2
('months', 'aapl') 0.08333333333333333
('best', 'iphone') 0.08695652173913043
('iphone', 'apps') 0.004651162790697674
('apps', 'year') 0.02857142857142857
('year', 'according') 0.03125
('apple', 'monday') 0.0005617977528

('apple', 'tree') 0.0005617977528089888
('tree', 'apple') 0.07142857142857142
('inc', 'leads') 0.009009009009009009
('leads', 'momentum') 0.5
('momentum', 'stocks') 0.5
('stocks', 'tesla') 0.03571428571428571
('tesla', 'motors') 1.0
('motors', 'inc') 1.0
('inc', 'loses') 0.009009009009009009
('loses', 'steam') 0.3333333333333333
('steam', 'httptcokcsz') 1.0
('httptcokcsz', 'aapl') 1.0
('aapl', 'expected') 0.001893939393939394
('expected', 'users') 0.3333333333333333
('users', 'still') 0.08333333333333333
('still', 'like') 0.034482758620689655
('like', 'small') 0.018518518518518517
('small', 'screens') 0.5
('screens', 'iphone') 0.3333333333333333
('mini', 'reportedly') 0.05
('reportedly', 'way') 0.2
('way', 'aapl') 0.02857142857142857
('httptcoh', 'five') 0.037037037037037035
('predictions', 'disagree') 0.5
('disagree', 'absolutely') 1.0
('absolutely', 'apple') 0.5
('apple', 'incredible') 0.0005617977528089888
('incredible', 'holiday') 0.5
('aapl', 'httptcogyjajevwoz') 0.001893939393939

('fix', 'real') 0.015873015873015872
('real', 'pain') 0.022727272727272728
('pain', 'us') 0.16666666666666666
('apple', 'care') 0.0005617977528089888
('care', 'cover') 0.125
('cover', 'free') 0.125
('free', 'aapliphone') 0.02127659574468085
('iphone', 'coming') 0.004651162790697674
('coming', 'styleshttptcoxbmuvdzpl') 0.14285714285714285
('styleshttptcoxbmuvdzpl', 'yeah') 0.5
('yeah', 'ill') 0.14285714285714285
('ill', 'definitely') 0.009345794392523364
('definitely', 'going') 0.5
('going', 'apple') 0.038461538461538464
('store', 'finals') 0.011764705882352941
('finals', 'damn') 0.5
('damn', 'mbp') 0.1
('mbp', 'graphics') 1.0
('graphics', 'card') 0.25
('card', 'failing') 0.058823529411764705
('failing', 'bullshit') 0.5
('bullshit', 'computer') 0.2
('computer', 'headed') 0.034482758620689655
('headed', 'ikea') 1.0
('ikea', 'apple') 1.0
('store', 'today') 0.011764705882352941
('today', 'yikes') 0.03571428571428571
('yikes', 'yay') 1.0
('yay', 'dayoffduties') 0.3333333333333333
('dayoffdu

('cons', 'getting') 0.05263157894736842
('getting', 'another') 0.09090909090909091
('another', 'crappy') 0.14285714285714285
('crappy', 'apple') 0.25
('product', 'ipad') 0.024390243902439025
('ipad', 'dont') 0.017241379310344827
('dont', 'update') 0.02127659574468085
('update', 'buggy') 0.01818181818181818
('buggy', 'slow') 0.3333333333333333
('apple', 'yay') 0.0005617977528089888
('yay', 'apple') 0.3333333333333333
('customer', 'support') 0.03125
('support', 'super') 0.041666666666666664
('super', 'nice') 0.5
('nice', 'helpful') 0.08333333333333333
('helpful', 'imessages') 0.5
('imessages', 'verified') 0.5
('verified', 'actual') 1.0
('heck', 'isnt') 0.06666666666666667
('isnt', 'taco') 0.058823529411764705
('taco', 'emoji') 0.25
('phone', 'acid') 0.0028011204481792717
('acid', 'right') 1.0
('right', 'apple') 0.05
('pls', 'let') 0.14285714285714285
('let', 'know') 0.010416666666666666
('know', 'yr') 0.034482758620689655
('yr', 'sorti') 0.05555555555555555
('sorti', 'problems') 0.333333

In [108]:
for row in df.text.values:
    doc = nlp(row)
    for chunk in doc.noun_chunks:
        freq = row.count(chunk.text)
        freq_text = text.count(chunk.text)
        df[chunk.text] = freq / freq_text

In [109]:
df1 = df.copy()

In [110]:
df1.set_index('text')

,y,snapchat damn time,apple inc httptcou aapl apple moneypress httptcowxkmmtmarw,u apple compile pics,one place,oddly uplifting story apple cofounder sold stake aapl aapl httptcopj,apple exchange,iphone different color lmao,mind,rt jpdesloges apple,...,whoabiebz,slack fuck boys,iphone mail,umo_games apple,work computer heres,vid cards,yearold wizkid,hollywood star httptcopoqfgoevsu,cycle,unit sales
text,,,,,,,,,,,,,,,,,,,,,
wow yall needa step apple rt heynyla music snapchat damn time thank,0.00125,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.125,0.071429,...,0.5,1.0,1.0,0.5,1.0,1.0,1.0,1.0,0.5,1.0
happened apple inc httptcou aapl apple moneypress httptcowxkmmtmarw,0.00125,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.125,0.071429,...,0.5,1.0,1.0,0.5,1.0,1.0,1.0,1.0,0.5,1.0
thank u apple compile pics communicate one place httptcooya,0.00125,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.125,0.071429,...,0.5,1.0,1.0,0.5,1.0,1.0,1.0,1.0,0.5,1.0
oddly uplifting story apple cofounder sold stake aapl aapl httptcopj,0.00125,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.125,0.071429,...,0.5,1.0,1.0,0.5,1.0,1.0,1.0,1.0,0.5,1.0
apple exchange iphone different color lmao changed mind,0.00125,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.125,0.071429,...,0.5,1.0,1.0,0.5,1.0,1.0,1.0,1.0,0.5,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pick slack fuck boys apple,0.00125,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.125,0.071429,...,0.5,1.0,1.0,0.5,1.0,1.0,1.0,1.0,0.5,1.0
finally got iphone mail comes box busted screen thanks apple,0.00125,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.125,0.071429,...,0.5,1.0,1.0,0.5,1.0,1.0,1.0,1.0,0.5,1.0
umo_games apple ended getting new computer anyhow work computer heres switch vid cards httptcoo,0.00125,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.125,0.071429,...,0.5,1.0,1.0,0.5,1.0,1.0,1.0,1.0,0.5,1.0


# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [111]:
df.drop(df.columns[1:], axis=1, inplace=True)
df

,text
0,wow yall needa step apple rt heynyla music sna...
1,happened apple inc httptcou aapl apple moneypr...
2,thank u apple compile pics communicate one pla...
3,oddly uplifting story apple cofounder sold sta...
4,apple exchange iphone different color lmao cha...
...,...
1625,pick slack fuck boys apple
1626,finally got iphone mail comes box busted scree...
1627,umo_games apple ended getting new computer any...
1628,yearold wizkid turned apple nabbed backing hol...


In [112]:
my_review = 'I love the IPhone. it is of best quality.'

In [113]:
my_review = my_review.lower()

In [114]:
my_review = my_review.replace('.', '')

In [115]:
my_review = ' '.join(word for word in my_review.split(' ') if word not in stop)

In [116]:
df.loc[-1] = my_review
df.index = df.index + 1
df = df.sort_index()

In [117]:
df

,text
0,love iphone best quality
1,wow yall needa step apple rt heynyla music sna...
2,happened apple inc httptcou aapl apple moneypr...
3,thank u apple compile pics communicate one pla...
4,oddly uplifting story apple cofounder sold sta...
...,...
1626,pick slack fuck boys apple
1627,finally got iphone mail comes box busted scree...
1628,umo_games apple ended getting new computer any...
1629,yearold wizkid turned apple nabbed backing hol...


In [118]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(list(df.text.values))

In [119]:
df_tfidf = pd.DataFrame.sparse.from_spmatrix(tfidf, columns=vectorizer.get_feature_names())

In [120]:
from sklearn.metrics.pairwise import linear_kernel
cosine_similarities = linear_kernel(tfidf[0:1], tfidf).flatten()
cosine_similarities

array([1.        , 0.        , 0.        , ..., 0.        , 0.        ,
       0.04688916])

In [121]:
related_docs_indices = cosine_similarities.argsort()[::-1]
related_docs_indices

array([   0, 1016,  802, ...,  974,  975,  815], dtype=int64)

In [122]:
print('the most similar reviews based on cosine similarity in decreasin order')
print('-'*80, end='\n\n')
for doc in df.text.values[related_docs_indices[1:]]:
    print(doc, end='\n\n')

the most similar reviews based on cosine similarity in decreasin order
--------------------------------------------------------------------------------

macbook love today apple

ios fucked battery life iphone really starting wonder apple quality control anymore wish

best stevejobs apple aapl httptcog via

ganondaif quality bad webcam pics excuse apple

best iphone apps year according apple apple monday december httptcovnvazcnemp

apple chargers suck ass love break

apple needs china love apple watch aapl aapl httptco

love new apple tv spot httptcoixsje

love phone much thank based god apple

love apple mobilephone httptcoajvsqxgdn httptcoo

meet elevate apple apples pick best iphone app year monday december httptcok

fuck iphone apple

fuck apple iphone

love apple supporting hourofcode workshops httptcofnjnu

apple conclusion best client service ever

tim_cook therealjonyive iosissues problems apple issues safari want jobs quality control back httptcodi

love apple need upgrade soo

early movers big dltr thi aapl five amp stocks posting largest moves bell httptcogblmwzocqo

first apple logo designed cofounder ronald wayne drawing isaac newton reading book apple tree

rt jpdesloges gtat creditors want question apples jeff williams aapl aapl httptcolkh

hell lucys songs delete phone hatw itunesmusic apple

never ever buy apple itouch worthless

onerepublic apple thanks sharing pics awesome guys

apple going fix maps app navigation sucks face switch google maps

every young designer know legendary apple designer susan kare httptcobwc httptcog

phone died apple fuck

ipod replaced never set foot apple store world better place

market create videos students apple macbook httptcobprq via donorschoose

apple dont want money going africa fight aids problems need fight apple africa cnn cnn

isnt apple twitter httptcomrpe

apple tv fades away streaming video player market httptconmqcdkfnul

aapl might invest tsla increase rampd plus increase production multifolds ev cars su

# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each of the 500 review (if you collected the data from citeseer, you can think of other text classfication tasks instead of sentiment analysis) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


Answer: https://github.com/DhatriPriya/DhatriPriya_INFO5731_Fall2021/blob/main/train_evaluation_dataset.csv

### Question 3 work

In [144]:
# The GitHub link of your final csv file
df2=pd.read_csv('processed.csv',encoding = "ISO-8859-1")
df2.head(500)
stop = stopwords.words('english')
df2['text'] = df2['text'].apply(lambda x: x.replace(r'"', ''))
df2['text'] = df2['text'].apply(
    lambda x: x.replace(r'[\n\n\n\n\n\n\n\n\n\n  \n  \n    , \n  ', ''))
df2['text'] = df2['text'].apply(
    lambda x: x.replace(r'\n, \n  \n]', ''))
df2['text'] = df2['text'].apply(lambda x: x.lower())
df2.text = df2.text.apply(lambda x: re.sub(r'\w*\d', '', x))
df2.text = df2.text.str.replace('[^\w\s]', '')
df2.text = df2.text.apply(
    lambda x: ' '.join(x for x in x.split() if x not in stop))
# Link: 

C:\Users\stech\AppData\Local\Temp/ipykernel_8916/4168923001.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.text = df2.text.str.replace('[^\w\s]', '')


In [145]:
df2.drop(['sentiment', 'character_cnt','word_counts','characters_per_word','special_characters','num'], axis=1, inplace=True)


In [146]:
df2.head()

,text,result
0,wow yall needa step apple rt heynyla music sna...,Negative
1,happened apple inc httptcou aapl apple moneypr...,Neutral
2,thank u apple compile pics communicate one pla...,Positive
3,oddly uplifting story apple cofounder sold sta...,Neutral
4,apple exchange iphone different color lmao cha...,Neutral


In [147]:
df2.to_csv('train_evaluation_dataset.csv')